### Simulation of a simple refrigerator

1. Import the necessary packages

In [1]:
from tespy.networks import Network
from tespy.components import (Sink, Source, Valve, SimpleHeatExchanger, Compressor, CycleCloser)
from tespy.connections import Connection, Bus
from CoolProp.CoolProp import PropsSI

2. Create the `Network` for the refrigerator and set the units you want to use

In [2]:
refrigerator = Network(T_unit='C', p_unit='bar', h_unit='kJ / kg', m_unit='kg / s')

3. Set the fluids of the system (for easier access in the functions)

In [3]:
wf = "R600a"
wf_dict = {wf: 1}

4. Set the components of the system

In [4]:
val = Valve('throttling-valve')
eva = SimpleHeatExchanger('evaporator')
cond = SimpleHeatExchanger('condenser')
comp = Compressor('compressor')
cc = CycleCloser('cycle-closer')


5. Set the connections of the system and add them to the `Network`

In [5]:
# cXX = Connection(src_comp, 'src_port', dst_comp, 'dst_port', label='XX')
c01 = Connection(eva, 'out1', comp, 'in1', label='1')
c02 = Connection(comp, 'out1', cond, 'in1', label='2')
c02cc = Connection(cond, 'out1', cc, 'in1', label='2cc')
c03 = Connection(cc, 'out1', val, 'in1', label='3')
c04 = Connection(val, 'out1', eva, 'in1', label='4')

refrigerator.add_conns(c01, c02, c02cc, c03, c04)

6. Define the parameters and design variables

In [6]:
# Components
eva.set_attr(Q=150, pr=1)

comp.set_attr(eta_s=0.75)

cond.set_attr(pr=1)

# Connections
c03.set_attr(fluid=wf_dict)

p4_set = PropsSI("P", "Q", 1, "T", 273.15 - 20, wf) / 1e5  # saturation pressure at given temperature
c04.set_attr(p=p4_set)

h1_set = PropsSI("H", "P", p4_set*1e5, "T", 273.15 - 18, wf) / 1e3  # superheated state at given temperature and pressure
c01.set_attr(h=h1_set)

c02.set_attr(T=60)

h2cc_set = PropsSI("H", "Q", 0, "T", 273.15 + 40, wf) / 1e3  # 
c02cc.set_attr(h=h2cc_set)

7. Solve the system of equations

In [7]:
refrigerator.solve(mode='design')
refrigerator.print_results()


 iter  | residual   | progress   | massflow   | pressure   | enthalpy   | fluid      | component  
-------+------------+------------+------------+------------+------------+------------+------------
 1     | 6.77e+05   | 1 %        | 2.00e+00   | 3.72e+05   | 2.04e+05   | 0.00e+00   | 0.00e+00   
 2     | 3.59e+04   | 16 %       | 3.17e-02   | 4.37e+05   | 8.20e+03   | 0.00e+00   | 0.00e+00   
 3     | 7.34e+03   | 23 %       | 1.09e-18   | 1.30e+05   | 3.22e+03   | 0.00e+00   | 0.00e+00   
 4     | 3.02e+02   | 39 %       | 0.00e+00   | 5.02e+03   | 1.79e+02   | 0.00e+00   | 0.00e+00   
 5     | 3.83e-01   | 71 %       | 0.00e+00   | 5.53e+00   | 2.66e-01   | 0.00e+00   | 0.00e+00   
 6     | 4.64e-07   | 100 %      | 0.00e+00   | 6.01e-06   | 3.52e-07   | 0.00e+00   | 0.00e+00   
 7     | 1.17e-10   | 100 %      | 0.00e+00   | 7.99e-10   | 1.32e-10   | 0.00e+00   | 0.00e+00   
Total iterations: 7, Calculation time: 0.07 s, Iterations per second: 97.79

##### RESULTS (Valve) #####
+--

8. Add busses for heat and power flows

In [8]:
motor = Bus('motor')
motor.add_comps({'comp': comp, 'char': 0.95, 'base': 'bus'})

cooling = Bus('cooling heat')
cooling.add_comps({'comp': eva, 'base': 'bus'})

refrigerator.add_busses(motor, cooling)

9. Run again the simulation and calculate the coefficient of performance

In [9]:
refrigerator.solve(mode='design')

print('COP:', cooling.P.val / motor.P.val)


 iter  | residual   | progress   | massflow   | pressure   | enthalpy   | fluid      | component  
-------+------------+------------+------------+------------+------------+------------+------------
 1     | 1.46e-11   | 100 %      | 0.00e+00   | 1.82e-10   | 3.92e-12   | 0.00e+00   | 0.00e+00   
 2     | 1.46e-11   | 100 %      | 0.00e+00   | 1.82e-10   | 3.92e-12   | 0.00e+00   | 0.00e+00   
 3     | 1.46e-11   | 100 %      | 0.00e+00   | 1.82e-10   | 3.92e-12   | 0.00e+00   | 0.00e+00   
 4     | 1.17e-10   | 100 %      | 0.00e+00   | 2.54e-09   | 5.49e-11   | 0.00e+00   | 0.00e+00   
Total iterations: 4, Calculation time: 0.02 s, Iterations per second: 248.38
COP: 2.00292017480205
